In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime
import statistics
import seaborn as sns

In [ ]:
from finrl.apps import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.finrl_meta.data_processor import DataProcessor
from finrl.finrl_meta.data_processors.processor_yahoofinance import YahooFinanceProcessor
import sys
sys.path.append("../FinRL-Library")

In [ ]:
import random
import torch
def set_seed(seed_value):
    random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

set_seed(100)

In [ ]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list
        self.max_portfolio_value = 0        
        self.mdd = 0.0001


        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
#         print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
            
            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))           
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions) 
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])            
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value 
            # self.reward = (new_portfolio_value-self.initial_amount)/self.initial_amount
            now_return = (new_portfolio_value-self.initial_amount)/self.initial_amount
            ann_return = (1+now_return) ** (252/self.day) -1
            if self.max_portfolio_value <  new_portfolio_value :
                self.max_portfolio_value = new_portfolio_value
            now_dd = 1-(new_portfolio_value/self.max_portfolio_value)
            if self.mdd < now_dd:
                self.mdd = now_dd    
#             self.reward = ann_return+(1/self.mdd)
#             print(ann_return)
#             print("Step reward: ", self.reward)
#             self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        seed = 100
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [ ]:
def get_data(etfs):
    dp = YahooFinanceProcessor()
    df = dp.download_data(start_date = '2008-01-01',
                     end_date = '2021-12-31',
                     ticker_list = etfs, time_interval='1D')
    fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

    df = fe.preprocess_data(df)
    # add covariance matrix as states
    df=df.sort_values(['date','tic'],ignore_index=True)
    df.index = df.date.factorize()[0]

    cov_list = []
    return_list = []

    # look back is one year
    lookback=252
    for i in range(lookback,len(df.index.unique())):
        data_lookback = df.loc[i-lookback:i,:]
        price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
        return_lookback = price_lookback.pct_change().dropna()
        return_list.append(return_lookback)

        covs = return_lookback.cov().values 
        cov_list.append(covs)

    df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
    df = df.merge(df_cov, on='date')
    df = df.sort_values(['date','tic']).reset_index(drop=True)
    return df

In [ ]:
def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            print('enable dropout')
            m.train()

In [ ]:
def train_rl(df,start_date,end_date,initial_amount=1000000):

    
    train = data_split(df, start_date,end_date)
    stock_dimension = len(train.tic.unique())
    state_space = stock_dimension
    print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
    
    env_kwargs = {
        "hmax": 100, 
        "initial_amount": initial_amount, 
        "transaction_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4
    }
    
    e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)
    env_train, _ = e_train_gym.get_sb_env()
    
    agent = DRLAgent(env = env_train)
    PPO_PARAMS = {
        "n_steps": 2048,
        "ent_coef": 0.005,
        "learning_rate": 0.0001,
        "batch_size": 128,
    }
    model_ppo = agent.get_model("ppo",seed=100,model_kwargs = PPO_PARAMS)
#     enable_dropout(model_ppo)
    trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=100000)
#     trained_ppo.save('trained_models/trained_ppo.zip')
    return trained_ppo

In [ ]:
def trade_rl(df,start_date,end_date,trained_model,initial_amount=1000000):
    trade = data_split(df,start_date,end_date)
    stock_dimension = len(trade.tic.unique())
    state_space = stock_dimension
    env_kwargs = {
        "hmax": 100, 
        "initial_amount": initial_amount, 
        "transaction_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4
    }
    e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)
    df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_model, environment = e_trade_gym)
    return df_daily_return, df_actions

In [ ]:
org_etfs = ['ITOT', 'VEU', 'VNQ', 'AGG']
org_df = get_data(org_etfs)

In [ ]:
org_df

In [ ]:
org_etfs = ['ITOT', 'VEU', 'VNQ', 'AGG']
ttt_org_df = pd.read_csv('process_data_classic.csv')
ttt_org_df.drop(["Unnamed: 0"], axis=1, inplace=True)
ttt_org_df

In [ ]:
org_df.to_csv('process_data_classic.csv')

In [ ]:
org_train_start = '2009-01-01'
org_train_end = '2016-01-01'
org_initial_amount = 1000000
models = []
for i in range(1):
    print(i)
    trained_model = train_rl(org_df,org_train_start,org_train_end,org_initial_amount)
    models.append(trained_model)

In [ ]:
org_trade_start = '2016-01-01' 
org_trade_end = '2021-01-01'
trade_action_list = []
trade_return_list = []
for i in range(1):
    trained_model = models[i]
#     train_df_daily_return, train_df_actions = trade_rl(org_df,org_train_start,org_train_end,trained_model,org_initial_amount)
    trade_df_daily_return, trade_df_actions = trade_rl(org_df,org_trade_start,org_trade_end,trained_model,org_initial_amount)
    trade_action_list.append(trade_df_actions)
    trade_return_list.append(trade_df_daily_return)

In [ ]:
for i in range(5):
    print(trade_action_list[i][1050:])
    print(trade_return_list[i][1050:])

In [ ]:
from pyfolio import timeseries
import pyfolio
%matplotlib inline

baseline_df = get_baseline(
        ticker="^DJI", 
        start = trade_return_list[0].loc[0,'date'],
        end = trade_return_list[0].loc[len(trade_return_list[0])-1,'date'])
baseline_returns = get_daily_return(baseline_df, value_col_name="close")

for i in range(1):
    DRL_strat = convert_daily_return_to_pyfolio_ts(trade_return_list[i])
    perf_func = timeseries.perf_stats 
    perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")
    print("==============DRL Strategy Stats===========")
#     print(perf_stats_all)
    
    with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(returns = DRL_strat,
                                       benchmark_rets=baseline_returns, set_context=False)

In [ ]:
train_df_daily_return, train_df_actions = trade_rl(org_df,'2011-01-01','2016-01-01',trained_model,org_initial_amount)

from pyfolio import timeseries
import pyfolio
%matplotlib inline

baseline_df = get_baseline(
        ticker="^DJI", 
        start = train_df_daily_return.loc[0,'date'],
        end = train_df_daily_return.loc[len(train_df_daily_return)-1,'date'])
baseline_returns = get_daily_return(baseline_df, value_col_name="close")

for i in range(1):
    DRL_strat = convert_daily_return_to_pyfolio_ts(train_df_daily_return)
    perf_func = timeseries.perf_stats 
    perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")
    print("==============DRL Strategy Stats===========")
#     print(perf_stats_all)
    
    with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(returns = DRL_strat,
                                       benchmark_rets=baseline_returns, set_context=False)

In [ ]:
df_daily_return_nnn, df_actions_nnn = trade_rl(org_df,'2021-01-01','2022-01-01',trained_model,org_initial_amount)

from pyfolio import timeseries
import pyfolio
%matplotlib inline

baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_daily_return_nnn.loc[0,'date'],
        end = df_daily_return_nnn.loc[len(df_daily_return_nnn)-1,'date'])
baseline_returns = get_daily_return(baseline_df, value_col_name="close")

for i in range(1):
    DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return_nnn)
    perf_func = timeseries.perf_stats 
    perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")
    print("==============DRL Strategy Stats===========")
#     print(perf_stats_all)
    
    with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(returns = DRL_strat,
                                       benchmark_rets=baseline_returns, set_context=False)

In [ ]:
vect_dict = {}

for j in range(len(trade_action_list[0])):
    tmp = []
    d = trade_action_list[0].reset_index()['date'][j]
    for i in range(5):
        trade_return_df = trade_action_list[i].reset_index().T
#         print(trade_return_df)
        vect = list(trade_return_df[j])[1:]
#         print(vect)
        tmp.append(vect)
    vect_dict[d] = tmp
    
# vect_dict


In [ ]:
import statistics
tmp = np.array(vect_dict['2016-01-05']).T.tolist()
for t in tmp:
    print(statistics.stdev(t))

In [ ]:
import statistics
stdev_dict = {}
for d in vect_dict:
    tmp = np.array(vect_dict[d]).T.tolist()
    ttt = []
    for t in tmp:
        ttt.append(statistics.stdev(t))
    avg_stdev = sum(ttt)/len(ttt)
    stdev_dict[d] = avg_stdev
#     print(ttt,avg_stdev)
stdev_list = list(stdev_dict.values())
# print(stdev_list)
plt.plot(stdev_list)
plt.show()

In [ ]:
from itertools import combinations
# from math import dist

diff_dict = {}
max_diff_dict = {}
mean_diff_dict = {}
for d in vect_dict:
    tmp = vect_dict[d]
    comb = list(combinations(tmp, 2))
    max_dist = 0
    tmp = []
    for c in comb:
        a = np.array(c[0])
        b = np.array(c[1])
        distance = np.sqrt(np.sum(np.square(a-b)))
#         distance = dist(c[0],c[1])
        if distance>max_dist:
            max_dist = distance
        tmp.append(distance)
    diff_dict[d] = tmp
    mean_diff_dict[d] = sum(tmp)/len(tmp)
    max_diff_dict[d] = max_dist
max_diff_list = list(max_diff_dict.values())
plt.plot(max_diff_list)
plt.show()

In [ ]:
count = 0
for m in max_diff_list:
    if m > 0.4:
        count+=1
print(count,len(max_diff_list))

In [ ]:
print(max(max_diff_list), min(max_diff_list) )

In [ ]:
max_diff_df = pd.DataFrame(list(max_diff_dict.items()),columns=['date','diff'] )
# max_diff_df[max_diff_df['diff']>0.4 ]
max_diff_df.to_csv('diff.csv')

In [ ]:
mean_diff_df = pd.DataFrame(list(mean_diff_dict.items()),columns=['date','diff'] )
mean_diff_df.to_csv('diff_mean.csv')

In [ ]:
for i in range(5):
    trade_action_list[i].to_csv('trade_action_'+str(i)+'.csv')
    trade_return_list[i].to_csv('trade_return_'+str(i)+'.csv')
    

### 看單一agent的輸出分布

In [ ]:
# org_trade_start = '2016-01-01' 
# org_trade_end = '2021-01-01'
org_trade_start = '2009-01-01'
org_trade_end = '2016-01-01'
trade_action_list = []
trade_return_list = []
trained_model = models[0]
for i in range(250):
    print((i))
    trade_df_daily_return, trade_df_actions = trade_rl(org_df,org_trade_start,org_trade_end,trained_model,org_initial_amount)
    trade_action_list.append(trade_df_actions)
    trade_return_list.append(trade_df_daily_return)

In [ ]:
vect_dict = {}

for j in range(len(trade_action_list[0])):
    tmp = []
    d = trade_action_list[0].reset_index()['date'][j]
    print(d)
    for i in range(250):
        trade_return_df = trade_action_list[i].reset_index().T
#         print(trade_return_df)
        vect = list(trade_return_df[j])[1:]
#         print(vect)
        tmp.append(vect)
    vect_dict[d] = tmp

tmp = np.array(vect_dict['2016-01-05']).T.tolist()
for t in tmp:
    print(statistics.stdev(t),sum(t)/len(t))

In [ ]:
# stdev = np.array(stdev_list).T[0]
# tmp = np.array(vect_dict['2016-01-05']).T.tolist()
for d in vect_dict:
    print(d)
    tmp = np.array(vect_dict[d]).T.tolist()
    fig,ax=plt.subplots(1,4,figsize=(30,8))
    for i in range(len(tmp)):
        # print(tmp[i])
        print(statistics.stdev(tmp[i][:]),sum(tmp[i][:])/len(tmp[i][:]))
        # ax[i].hist(tmp[i], density=False, color = 'lightblue', cumulative = False, bins=100)
#         ax2[i].boxplot(tmp[i])
        sns.distplot(tmp[i][:],ax = ax[i],bins=10 )#,bins=25
    plt.show()
    plt.close()


In [ ]:
stdev_dict = {}
mean_dict = {}
detect = []
for d in vect_dict:
    tmp = np.array(vect_dict[d]).T.tolist()
    ttt1 = []
    ttt2 = []
    for i in range(len(tmp)):
        t = tmp[i]
        stdev = statistics.stdev(t[:])
        mean = sum(t[:])/len(t[:])
        ttt1.append(stdev)#每天4股的stdev
        ttt2.append(mean) #mean
        if stdev>0.10:
#         if stdev<0.08:
            detect.append([d,org_etfs[i],stdev,mean])
#     avg_stdev = sum(ttt)/len(ttt)
#     stdev_dict[d] = avg_stdev
#     print(ttt,avg_stdev)
    stdev_dict[d] = ttt1
    mean_dict[d] = ttt2
stdev_list = list(stdev_dict.values())
mean_list = list(mean_dict.values())
# print(stdev_list)
plt.plot(stdev_list)
plt.show()
plt.plot(mean_list)
plt.show()
print(detect)

In [ ]:
transp_stdev_list = np.array(stdev_list).T
sig = statistics.stdev(transp_stdev_list.reshape(-1))
mu = sum(transp_stdev_list.reshape(-1))/len(transp_stdev_list.reshape(-1))
print(sig,mu)
print(sig*2+mu,sig*2.5+mu,sig*3+mu)#2.3 0.5 0.15%

In [ ]:
stdev_dict = {}
mean_dict = {}
detect = []
for d in vect_dict:
    tmp = np.array(vect_dict[d]).T.tolist()
    ttt1 = []
    ttt2 = []
    for i in range(len(tmp)):
        t = tmp[i]
        stdev = statistics.stdev(t[:])
        mean = sum(t[:])/len(t[:])
        ttt1.append(stdev)#每天4股的stdev
        ttt2.append(mean) #mean
        if stdev>0.10:
#         if stdev<0.08:
            detect.append([d,org_etfs[i],stdev,mean])
#     avg_stdev = sum(ttt)/len(ttt)
#     stdev_dict[d] = avg_stdev
#     print(ttt,avg_stdev)
    stdev_dict[d] = ttt1
    mean_dict[d] = ttt2
stdev_list = list(stdev_dict.values())
mean_list = list(mean_dict.values())
# print(stdev_list)
plt.plot(stdev_list)
plt.show()
plt.plot(mean_list)
plt.show()
print(detect)

In [ ]:
transp_stdev_list = np.array(stdev_list).T
t1 = []
t2 = []
for i in range(len(transp_stdev_list)):
    t1.append(statistics.stdev(transp_stdev_list[i]))
    t2.append(sum(transp_stdev_list[i])/len(transp_stdev_list[i]))
    print(statistics.stdev(transp_stdev_list[i]),sum(transp_stdev_list[i])/len(transp_stdev_list[i]))
sig = sum(t1)/len(t1)
mu = sum(t2)/len(t2)
print(sig,mu,mu+2.5*sig)

In [ ]:
sig = statistics.stdev(transp_stdev_list.reshape(-1))
mu = sum(transp_stdev_list.reshape(-1))/len(transp_stdev_list.reshape(-1))
print(sig,mu)
print(sig*2+mu,sig*2.5+mu,sig*3+mu)#2.3 0.5 0.15%

In [ ]:
for i in range(4):
    print(sum(np.array(stdev_list).T[i])/len(np.array(stdev_list).T[i]))
    plt.plot(np.array(stdev_list).T[i] )
    plt.show()

In [ ]:
for i in range(4):
    print(sum(np.array(mean_list).T[i])/len(np.array(mean_list).T[i]))
    plt.plot(np.array(mean_list).T[i])
    plt.show()

### 用google trend找一些時間點

In [ ]:
### https://www.itread01.com/content/1584432422.html
from pytrends.request import TrendReq
from pprint import pprint


pytrend = TrendReq(hl='en-US', tz=360)
keywords = ['Coronavirus', 'Covid19','Covid-19','Sars']
pytrend.build_payload(
     kw_list=keywords,
     cat=0,
     timeframe='2020-02-01 2020-03-31', #today 1-m
     geo='US',
     gprop='')

pprint(pytrend.interest_over_time())

In [ ]:
for i in range(4):
    print(sum(np.array(stdev_list).T[i])/len(np.array(stdev_list).T[i]))
    for j in range(5):
        plt.plot(np.array(stdev_list).T[i][j*252:(j+1)*252] )
        plt.show()

In [ ]:

tmp = np.array(vect_dict['2016-01-05']).T.tolist()
fig1,ax1=plt.subplots(1,4,figsize=(30,8))
# fig2,ax2=plt.subplots(1,4,figsize=(30,8))
for i in range(len(tmp)):
#     ax1[i].boxplot(tmp[i],vert=False,showmeans=True)
#     ax1[i].hist(tmp[i], density=False, color = 'lightblue', cumulative = False, bins=100)
    sns.distplot(tmp[i],ax = ax1[i] ,bins=25)
plt.show()

In [ ]:
sns.distplot(tmp[0])

In [ ]:
np.array(vect_dict['2016-01-05']).T.tolist()

In [ ]:
trade_action_list[0].reset_index()

In [ ]:

class DRLAgentEnsembleNew:
    """Provides implementations for DRL algorithms

    Attributes
    ----------
        env: gym environment class
            user-defined class

    Methods
    -------
        get_model()
            setup DRL algorithms
        train_model()
            train DRL algorithms in a train dataset
            and output the trained model
        DRL_prediction()
            make a prediction in a test dataset and get results
    """

    def __init__(self, env):
        self.env = env

    def get_model(
        self,
        model_name,
        policy="MlpPolicy",
        policy_kwargs=None,
        model_kwargs=None,
        verbose=1,
        seed=None,
        tensorboard_log=None,
    ):
        if model_name not in MODELS:
            raise NotImplementedError("NotImplementedError")

        if model_kwargs is None:
            model_kwargs = MODEL_KWARGS[model_name]

        if "action_noise" in model_kwargs:
            n_actions = self.env.action_space.shape[-1]
            model_kwargs["action_noise"] = NOISE[model_kwargs["action_noise"]](
                mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions)
            )
        print(model_kwargs)
        model = MODELS[model_name](
            policy=policy,
            env=self.env,
            tensorboard_log=tensorboard_log,
            verbose=verbose,
            policy_kwargs=policy_kwargs,
            seed=seed,
            **model_kwargs,
        )
        return model

    def train_model(self, model, tb_log_name, total_timesteps=5000):
        model = model.learn(
            total_timesteps=total_timesteps,
            tb_log_name=tb_log_name,
            callback=TensorboardCallback(),
        )
        return model

    @staticmethod
    def DRL_prediction(model, environment):
        test_env, test_obs = environment.get_sb_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []
        test_env.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = model.predict(test_obs)
            # account_memory = test_env.env_method(method_name="save_asset_memory")
            # actions_memory = test_env.env_method(method_name="save_action_memory")
            test_obs, rewards, dones, info = test_env.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = test_env.env_method(method_name="save_asset_memory")
                actions_memory = test_env.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break
        return account_memory[0], actions_memory[0]
    
    def run_ensemble_strategy(self, df, model_kwargs, timesteps):
        stock_dimension = len(df.tic.unique())
        state_space = stock_dimension
        e_train_gym = StockPortfolioEnv(df = df, model_kwargs)
        env_train, _ = e_train_gym.get_sb_env()
        self.env = env_train
        PPO_PARAMS = {
            "n_steps": 2048,
            "ent_coef": 0.005,
            "learning_rate": 0.0001,
            "batch_size": 128,
        }
        for i in range(10):
            model_ppo = self.get_model("ppo",model_kwargs = PPO_PARAMS)
            model_ppo = self.train_model(model=model_ppo, tb_log_name='ppo', total_timesteps=100000)
            df_daily_return, df_actions = self.DRL_prediction(model=model_ppo, environment = e_train_gym)
